In [1]:
# Importando os módulos que serão utilizados

import tensorflow as tf
import numpy as np
import tensorflow.keras as keras
import time
import dill
import pickle
import pandas as pd
import plotly.express as ply_exp
import plotly.graph_objects as ply_go

# Configurando o acesso do tensorflow aos processadores do computador
# no caso, está sendo selecionada a primeira placa de vídeo listada

# gpus = tf.config.experimental.list_physical_devices('GPU') # Listando as placas de vídeo
# tf.config.experimental.set_memory_growth(gpus[0], True)    # Selecionando a primeira GPU e configurando

# Importando os módulos contendo as funções criadas no projeto

from aux_func_V3 import *
import API_Numpy
import API_TensorFlow

In [2]:

# nomes = [
#     "WENO-JS", "WENO-JS (M)", "WENO-JS (MS)", "WENO-JS (BI)",
#     "WENO-Z" , "WENO-Z (M)" , "WENO-Z (MS)" , "WENO-Z (BI)" , 
#     "WENO-Z+", "WENO-Z+ (M)", "WENO-Z+ (MS)", "WENO-Z+ (BI)",
#     "WENO-ZC (M)", "WENO-ZC (MS)", "WENO-ZC (BI)"
# ]
# cores = [
#     "273EAF", "5B7BD7", "8EA9F1", "BEC7FF", # WENO-JS: Azul
#     "A41127", "DD5C5B", "FF8891", "FFC2D0", # WENO-Z : Vermelho
#     "A45705", "D26F1C", "F2944A", "FFC29A", # WENO-Z+: Laranja
#     "41631D", "5B9940", "9CD87B", "DBF7C1"  # WENO-ZC: Verde
# ]


# API      = API_Numpy
# equation = euler_equation
# γ        = 1.4

# WENOs = {}

# WENOs[nomes[ 0]] = WENO_JS(   API, equation, γ=γ).Sim
# WENOs[nomes[ 1]] = WENO_JS_M( API, equation, γ=γ).Sim
# WENOs[nomes[ 2]] = WENO_JS_MS(API, equation, γ=γ).Sim
# WENOs[nomes[ 3]] = WENO_JS_BI(API, equation, γ=γ).Sim

# WENOs[nomes[ 4]] = WENO_Z(   API, equation, γ=γ).Sim
# WENOs[nomes[ 5]] = WENO_Z_M( API, equation, γ=γ).Sim
# WENOs[nomes[ 6]] = WENO_Z_MS(API, equation, γ=γ).Sim
# WENOs[nomes[ 7]] = WENO_Z_BI(API, equation, γ=γ).Sim

# WENOs[nomes[ 8]] = WENO_Zp(   API, equation, γ=γ).Sim
# WENOs[nomes[ 9]] = WENO_Zp_M( API, equation, γ=γ).Sim
# WENOs[nomes[10]] = WENO_Zp_MS(API, equation, γ=γ).Sim
# WENOs[nomes[11]] = WENO_Zp_BI(API, equation, γ=γ).Sim

# WENOs[nomes[12]] = WENO_ZC_M( API, equation, γ=γ).Sim
# WENOs[nomes[13]] = WENO_ZC_MS(API, equation, γ=γ).Sim
# WENOs[nomes[14]] = WENO_ZC_BI(API, equation, γ=γ).Sim

# nomes = [
#     "WENO-Z" , "WENO-Z (BIm)" , "WENO-Z (BI)" , "WENO-Z (BIM)" ,
#     "WENO-Z+", "WENO-ZC (BIm)", "WENO-ZC (BI)", "WENO-ZC (BIM)"
# ]
# cores = [
#     "#A41127", "#DD5C5B", "#FF8891", "#FFC2D0", # WENO-Z : Vermelho
#     "#273EAF", "#5B7BD7", "#8EA9F1", "#BEC7FF"  # WENO-ZC: Azul
# ]

# API      = API_Numpy
# equation = euler_equation
# γ        = 1.4

# WENOs = {}

# WENOs[nomes[ 0]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 1]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 2]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 3]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BIM_mapping , network=None, p=2, ε=ε_default).Sim

# WENOs[nomes[ 4]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=null_mapping   , map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 5]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 6]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 7]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BIM_mapping , network=None, p=2, ε=ε_default).Sim

# nomes = [
#     "WENO-Z" , "WENO-Z (BIm)" , "WENO-Z (BI)" , "WENO-Z (ENO)" ,
#     "WENO-Z+", "WENO-ZC (BIm)", "WENO-ZC (BI)", "WENO-ZC (ENO)"
# ]
# cores = [
#     "#A41127", "#DD5C5B", "#FF8891", "#FFC2D0", # WENO-Z : Vermelho
#     "#273EAF", "#5B7BD7", "#8EA9F1", "#BEC7FF"  # WENO-ZC: Azul
# ]

# API      = API_Numpy
# equation = euler_equation
# γ        = 1.4

# WENOs = {}

# WENOs[nomes[ 0]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 1]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 2]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 3]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=ENO_mapping , network=None, p=2, ε=ε_default).Sim

# WENOs[nomes[ 4]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=null_mapping   , map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 5]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 6]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 7]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=ENO_mapping , network=None, p=2, ε=ε_default).Sim

# nomes = [
#     "WENO-JS", "WENO-JS (MS)", "WENO-JS (BIm)", "WENO-JS (BI)",
#     "WENO-Z" , "WENO-Z (MS)" , "WENO-Z (BIm)" , "WENO-Z (BI)" , 
#     "WENO-Z+", "WENO-ZC (MS)", "WENO-ZC (BIm)", "WENO-ZC (BI)"
# ]
# cores = [
#     "#273EAF", "#5B7BD7", "#8EA9F1", "#BEC7FF", # WENO-JS: Azul
#     "#A41127", "#DD5C5B", "#FF8891", "#FFC2D0", # WENO-Z : Vermelho
#     "#41631D", "#5B9940", "#9CD87B", "#DBF7C1"  # WENO-ZC: Verde
# ]

# API      = API_Numpy
# equation = euler_equation
# γ        = 1.4

# WENOs = {}

# WENOs[nomes[ 0]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 1]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=pre_mapping , map_function=Hong_mapping, network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 2]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=pre_mapping , map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 3]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=pre_mapping , map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim

# WENOs[nomes[ 4]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 5]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=Hong_mapping, network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 6]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 7]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim

# WENOs[nomes[ 8]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=null_mapping   , map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 9]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=Hong_mapping, network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[10]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[11]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim


# nomes = [
#     "WENO-Z", "WENO-Z (RK)"
# ]
# cores = [
#     "#273EAF", "#8EA9F1"  # WENO-Z: Azul
# ]

# API         = API_Numpy
# equation    = euler_equation
# equation_RK = euler_equation_RK
# γ           = 1.4

# WENOs = {}

# WENOs[nomes[ 0]] = simulation(   API, equation   , WENO_Z_scheme   , γ, mapping=null_mapping   , map_function=lambda x:x, network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 1]] = simulation_RK(API, equation_RK, WENO_Z_RK_scheme, γ, mapping=null_mapping_RK, map_function=lambda x:x, network=None, p=2, ε=ε_default).Sim


# nomes = [
#     "WENO-JS", "WENO-JS (MS)", "WENO-JS (BI)", "WENO-JS (BIm)",
#     "WENO-Z" , "WENO-Z (MS)" , "WENO-Z (BI)" , "WENO-Z (BIm)" ,
#     "WENO-Z+", "WENO-ZC (MS)", "WENO-ZC (BI)", "WENO-ZC (BIm)",
#     "WENO-D" , "WENO-A"      #, "WENO-JS (ENO)", "WENO-Z (ENO)"
# ]
# cores = [
#     "#303387", "#4262C1", "#7787D9", "#ABB6FF", # WENO-JS: Azul
#     "#892937", "#C23C52", "#DB6579", "#FF8FAB", # WENO-Z : Vermelho
#     "#5A2571", "#801AA2", "#A23CB1", "#C66ABE", # WENO-ZC: Roxo
#     "#3A561F", "#487135", "#57933B", "#83B35E"  # WENO-D : Verde
# ] # Cores atualizadas

# API      = API_Numpy
# equation = euler_equation
# γ        = 1.4

# WENOs = {}

# WENOs[nomes[ 0]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 1]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=pre_mapping , map_function=Hong_mapping, network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 2]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=pre_mapping , map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 3]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=pre_mapping , map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim

# WENOs[nomes[ 4]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 5]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=Hong_mapping, network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 6]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 7]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping , map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim

# WENOs[nomes[ 8]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=null_mapping   , map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[ 9]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=Hong_mapping, network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[10]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[11]] = simulation(API, equation, WENO_ZC_scheme, γ, mapping=pre_inv_mapping, map_function=BI_mapping  , network=None, p=2, ε=ε_default).Sim

# WENOs[nomes[12]] = simulation(API, equation, WENO_D_scheme, γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[13]] = simulation(API, equation, WENO_A_scheme, γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default).Sim

# WENOs[nomes[14]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=pre_mapping, map_function=ENO_mapping, network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[15]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=pre_mapping, map_function=ENO_mapping, network=None, p=2, ε=ε_default).Sim

#---------------------------------------------------------------------------------------------------------------------------------------------------------------
# nomes = ["WENO-JS", "WENO-Z" , "WENO-Z (MS)", "WENO-Z (BIm)", "WENO-D" , "WENO-Z (BI2)"]
# cores = ["#4262C1", "#C23C52", "#57933B"    , "#A23CB1"     , "#C2881D", "#72C5D6"     ] # Cores atualizadas

# API      = API_Numpy
# equation = euler_equation
# γ        = 1.4

# WENOs = {}
# WENOs[nomes[0]] = simulation(API, equation, WENO_JS_scheme, γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[1]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[2]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=pre_mapping , map_function=Hong_mapping, network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[3]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=pre_mapping , map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[4]] = simulation(API, equation, WENO_D_scheme , γ, mapping=null_mapping, map_function=lambda x:x  , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[5]] = simulation(API, equation, WENO_Z_scheme , γ, mapping=pre_mapping , map_function=BI2_mapping , network=None, p=2, ε=ε_default).Sim
#---------------------------------------------------------------------------------------------------------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------------------------------------------------------------
# nomes = ["WENO-Z (BI1)", "WENO-Z (BI2)", "WENO-Z (BI3)", "WENO-Z (BI4)", "WENO-Z (BIm)", "WENO-Z (MS)"]
# cores = ["#4262C1", "#C23C52", "#57933B", "#A23CB1", "#C2881D", "#72C5D6"] # Cores atualizadas

# API      = API_Numpy
# equation = euler_equation
# γ        = 1.4

# WENOs = {}
# WENOs[nomes[0]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping, map_function=BI1_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[1]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping, map_function=BI2_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[2]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping, map_function=BI3_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[3]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping, map_function=BI4_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[4]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping, map_function=BIm_mapping , network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[5]] = simulation(API, equation, WENO_Z_scheme, γ, mapping=pre_mapping, map_function=Hong_mapping, network=None, p=2, ε=ε_default).Sim
#---------------------------------------------------------------------------------------------------------------------------------------------------------------

#---------------------------------------------------------------------------------------------------------------------------------------------------------------
nomes = ["WENO-JS", "WENO-Z", "WENO-Z+", "WENO-ZD", "WENO-ZD+"]
# nomes = ["WENO-JS", "WENO-Z", "WENO-ZD1", "WENO-ZD2", "WENO-ZD3", "WENO-ZD4"]
cores = ["#4262C1", "#C23C52", "#57933B", "#A23CB1", "#C2881D", "#72C5D6"] # Cores atualizadas

API      = API_Numpy
equation = euler_equation
γ        = 1.4

WENOs = {}
WENOs[nomes[0]] = simulation(API, equation, WENO_JS_scheme , γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default).Sim
WENOs[nomes[1]] = simulation(API, equation, WENO_Z_scheme  , γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default).Sim
WENOs[nomes[2]] = simulation(API, equation, WENO_Zp_scheme , γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default).Sim
WENOs[nomes[3]] = simulation(API, equation, WENO_ZD_scheme , γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[3]] = simulation(API, equation, WENO_ZD2_scheme, γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[4]] = simulation(API, equation, WENO_ZD3_scheme, γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default).Sim
# WENOs[nomes[5]] = simulation(API, equation, WENO_ZD4_scheme, γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default).Sim
WENOs[nomes[4]] = simulation(API, equation, WENO_ZDp_scheme, γ, mapping=null_mapping, map_function=lambda x:x, network=None, p=2, ε=ε_default).Sim


# nomes = ["WENO-JS", "WENO-Z", "WENO-Z+", "WENO-ZD+"]
#---------------------------------------------------------------------------------------------------------------------------------------------------------------

In [3]:

def cond_inicial(problema):
    
    if problema == "1-Lax":
        
        # Riemann problem of Lax
        #-------------------------------------------------------------------
        t_final   = 0.13
        n         = 200
        interval  = [-0.5, 0.5]
        fator     = 40
        n_ref     = n*fator
        fronteira = FronteiraFixa
        #-------------------------------------------------------------------

    if problema == "2-Sod":

        # Riemann problem of Sod
        #-------------------------------------------------------------------
        t_final   = 2
        n         = 200
        interval  = [-5, 5]
        fator     = 40
        n_ref     = n*fator
        fronteira = FronteiraFixa
        #-------------------------------------------------------------------

    if problema == "3-Shock Density Wave Titarev-Toro":
        
        # Shock density wave Titarev–Toro
        #-------------------------------------------------------------------
        t_final   = 5
        n         = 1500
        interval  = [-10, 5]
        fator     = 8
        n_ref     = n*fator
        fronteira = TitarevToroBoundary
        #-------------------------------------------------------------------
        
    if problema == "4-Shock Density Wave Shu-Osher":
        
        # Shock density wave Shu–Osher
        #-------------------------------------------------------------------
        t_final   = 1.8
        n         = 200
        interval  = [-5, 5]
        fator     = 40
        n_ref     = n*fator
        fronteira = FronteiraFixa
        #-------------------------------------------------------------------
        
    if problema == "5-Blast Wave":
        
        # Blast wave
        #-------------------------------------------------------------------
        t_final   = 0.038
        n         = 400
        interval  = [0, 1]
        fator     = 8
        n_ref     = n*fator
        fronteira = FronteiraReflexiva
        #-------------------------------------------------------------------
        
    Δx     = (interval[1]-interval[0])/n
    Δx_ref = (interval[1]-interval[0])/n_ref

    x     = tf.range(interval[0], interval[1], Δx    , dtype=dtype)
    x_ref = tf.range(interval[0], interval[1], Δx_ref, dtype=dtype)

    CFL = 0.1
    γ   = 1.4

    if problema == "1-Lax":
        
        # Riemann problem of Lax
        #-------------------------------------------------------------------
        def f_test(x):
            ρ = np.where(x < 0, 0.445, 0.5)
            u = np.where(x < 0, 0.698, 0.0)
            p = np.where(x < 0, 3.528, 0.5710)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q

        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------

    if problema == "2-Sod":
        
        # Riemann problem of Sod
        #-------------------------------------------------------------------
        def f_test(x):
            ρ = np.where(x < 0, 0.125, 1)
            u = x-x
            p = np.where(x < 0, 0.1  , 1)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q
        
        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------

    if problema == "3-Shock Density Wave Titarev-Toro":
    
        # Shock density wave Titarev–Toro
        #-------------------------------------------------------------------
        def f_test(x):
            k = 20
            ρ = np.where(x < -4.5, 1.515695, 1+np.sin(k*np.pi*x)/10)
            u = np.where(x < -4.5, 0.523346, 0)
            p = np.where(x < -4.5, 1.805000, 1)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q
        
        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------

    if problema == "4-Shock Density Wave Shu-Osher":
                
        # Shock density wave Shu–Osher
        #-------------------------------------------------------------------
        def f_test(x):
            k = 5
            ρ = np.where(x < -4, 3.857143, 1+0.2*np.sin(k*x))
            u = np.where(x < -4, 2.629369, 0)
            p = np.where(x < -4, 31/3    , 1)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q
        
        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------

    if problema == "5-Blast Wave":
        
        # Blast wave
        #-------------------------------------------------------------------
        def f_test(x):
            ρ = x-x+1
            u = x-x
            p = np.where(  x < 0.1, 1000, 0.01)
            p = np.where(0.9 <   x,  100,    p)
            E = p/(γ-1)+ρ*(u**2)/2
            Q = np.stack([ρ,ρ*u,E], axis=0)
            return Q
        
        Q     = f_test(x)
        Q_ref = f_test(x_ref)
        #-------------------------------------------------------------------
    
    init = {
        "Q"        : Q         ,
        "Q_ref"    : Q_ref     ,
        "t_final"  : t_final   ,
        "Δx"       : Δx        ,
        "Δx_ref"   : Δx_ref    ,
        "CFL"      : CFL       ,
        "fronteira": fronteira ,
        "interval" : interval  ,
        "fator"    : fator
    }
    
    return init

In [4]:
problemas = [
    "1-Lax"                            ,
    "2-Sod"                            ,
    "3-Shock Density Wave Titarev-Toro",
    "4-Shock Density Wave Shu-Osher"   ,   
    "5-Blast Wave"
]

# problemas = ["3-Shock Density Wave Titarev-Toro"]
# problemas = ["4-Shock Density Wave Shu-Osher", "5-Blast Wave"]

# Carregando as soluções de referência
with open('htmls/referência', 'rb') as file:
    referencia = pickle.load(file)

# Carregando as soluções de cada método
with open('htmls/soluções', 'rb') as file:
    sol = pickle.load(file)

u = {}
for problema in problemas:
    
    init = cond_inicial(problema)
    
    Q         = init["Q"]
    Q_ref     = init["Q_ref"]
    t_final   = init["t_final"]
    Δx        = init["Δx"]
    Δx_ref    = init["Δx_ref"]
    CFL       = init["CFL"]
    fronteira = init["fronteira"]
    
    for nome in nomes: 
        
#         u[problema + ": " + nome] = WENOs[nome](Q, t_final, Δx, CFL, fronteira)
        
#-----------------------------------------------------------------------------------------------
        # Transferindo as soluções para cada método pré-calculadas
        try:
            u[problema + ": " + nome] = sol[problema + ": " + nome]
            print(problema[2:] + ": solução carregada para o " + nome)
        except:
            # Gerando as soluções para cada método
            try:
                tempo = time.time()
                u[problema + ": " + nome] = WENOs[nome](Q, t_final, Δx, CFL, fronteira)
                tempo = int(100*(time.time() - tempo))/100
                print(problema[2:] + ": solução obtida para o " + nome)
                print("Tempo total: " + str(tempo) + " segundo(s)")
            except:
                u[problema + ": " + nome] = Q[...]
                print(problema[2:] + ": NÃO passou o " + nome)
#-----------------------------------------------------------------------------------------------

#-----------------------------------------------------------------------------------------------
    # Transferindo as soluções de referência pre-calculadas
    try:
        u[problema + ": referência"] = referencia[problema + ": referência"]
        print(problema[2:] + ": referência foi carregada")
    except:
        # Gerando as soluções de referência
        try:
            u[problema + ": referência"] = WENOs["WENO-Z"](Q_ref, t_final, Δx_ref, 0.1, fronteira)
            print(problema[2:] + ": referência foi obtida")
        except:
            u[problema + ": referência"] = Q_ref[...]
            print(problema[2:] + ": referência NÃO foi obtida")
#-----------------------------------------------------------------------------------------------


Lax: solução carregada para o WENO-JS
Lax: solução carregada para o WENO-Z
Lax: solução carregada para o WENO-Z+
Lax: solução carregada para o WENO-ZD
Lax: solução carregada para o WENO-ZD+
Lax: referência foi carregada
Sod: solução carregada para o WENO-JS
Sod: solução carregada para o WENO-Z
Sod: solução carregada para o WENO-Z+
Sod: solução carregada para o WENO-ZD
Sod: solução carregada para o WENO-ZD+
Sod: referência foi carregada
Shock Density Wave Titarev-Toro: solução carregada para o WENO-JS
Shock Density Wave Titarev-Toro: solução carregada para o WENO-Z
Shock Density Wave Titarev-Toro: solução carregada para o WENO-Z+
Shock Density Wave Titarev-Toro: solução carregada para o WENO-ZD
Shock Density Wave Titarev-Toro: solução carregada para o WENO-ZD+
Shock Density Wave Titarev-Toro: referência foi carregada
Shock Density Wave Shu-Osher: solução carregada para o WENO-JS
Shock Density Wave Shu-Osher: solução carregada para o WENO-Z
Shock Density Wave Shu-Osher: solução carregada

In [6]:
df    = {}
x     = {}
y     = {}
label = {}

legend_position_x = {}
legend_position_y = {}
legend_anchor_x   = {}
legend_anchor_y   = {}

legend_position_x["1-Lax"] = "left"
legend_position_y["1-Lax"] = "top"

legend_position_x["2-Sod"] = "left"
legend_position_y["2-Sod"] = "top"

legend_position_x["3-Shock Density Wave Titarev-Toro"] = "left"
legend_position_y["3-Shock Density Wave Titarev-Toro"] = "bottom"

legend_position_x["4-Shock Density Wave Shu-Osher"] = "left"
legend_position_y["4-Shock Density Wave Shu-Osher"] = "bottom"

legend_position_x["5-Blast Wave"] = "left"
legend_position_y["5-Blast Wave"] = "top"

for problema in problemas:
    
    if legend_position_x[problema] == "left":
        legend_anchor_x[problema] = 0.01
        
    if legend_position_x[problema] == "right":
        legend_anchor_x[problema] = 0.99
        
    if legend_position_y[problema] == "bottom":
        legend_anchor_y[problema] = 0.01
        
    if legend_position_y[problema] == "top":
        legend_anchor_y[problema] = 0.99

for problema in problemas:
    
    init     = cond_inicial(problema)
    interval = init["interval"]
    Δx       = init["Δx"]
    
    if problema == "3-Shock Density Wave Titarev-Toro":
        x[problema]     = tf.range(interval[0], interval[1], Δx, dtype=dtype)[500:]
        y[problema]     = u[problema + ": " + nomes[0]][0][500:]
        label[problema] = tf.repeat([nomes[0]], [len(u[problema + ": " + nomes[0]][0][500:])])

        for i in tf.range(1, len(nomes)):
            x[problema]     = tf.concat([x[problema]    , tf.range(interval[0], interval[1], Δx, dtype=dtype)[500:]], axis = 0)
            y[problema]     = tf.concat([y[problema]    , u[problema + ": " + nomes[i]][0][500:]], axis = 0)
            label[problema] = tf.concat([label[problema], tf.repeat([nomes[i]], [len(u[problema + ": " + nomes[i]][0][500:])])], axis = 0)
    
    else:
        x[problema]     = tf.range(interval[0], interval[1], Δx, dtype=dtype)
        y[problema]     = u[problema + ": " + nomes[0]][0]
        label[problema] = tf.repeat([nomes[0]], [len(u[problema + ": " + nomes[0]][0])])

        for i in tf.range(1, len(nomes)):
            x[problema]     = tf.concat([x[problema]    , tf.range(interval[0], interval[1], Δx, dtype=dtype)], axis = 0)
            y[problema]     = tf.concat([y[problema]    , u[problema + ": " + nomes[i]][0]], axis = 0)
            label[problema] = tf.concat([label[problema], tf.repeat([nomes[i]], [len(u[problema + ": " + nomes[i]][0])])], axis = 0)

    
    df[problema] = pd.DataFrame(
        {
            "x"     : x[problema]     ,
            "y"     : y[problema]     ,
            "label" : label[problema]
        }
    )
    
    init     = cond_inicial(problema)
    interval = init["interval"]
    Δx_ref   = init["Δx_ref"]
    
    fig = ply_exp.line(
        data_frame = df[problema]    ,
        x          = x[problema]     ,
        y          = y[problema]     ,
        color      = label[problema] ,
        color_discrete_sequence = cores
    )
    
    fig.update_layout(
        paper_bgcolor = "#FFFFFF",
        plot_bgcolor  = "#FFFFFF",
        xaxis         = dict(
            linecolor = "#000000" , 
            ticks     = "inside"  ,
            title     = "x"
        ),
        yaxis         = dict(
            linecolor = "#000000" ,
            ticks     = "inside"  ,
            title     = "density"  
        ),
        legend = dict(
            title       = ""                          ,
            yanchor     = legend_position_y[problema] ,
            y           = legend_anchor_y[problema]   ,
            xanchor     = legend_position_x[problema] ,
            x           = legend_anchor_x[problema]   ,
            bordercolor = "#666666"                   ,
            borderwidth = 1
        )
    )
    
    if problema == "3-Shock Density Wave Titarev-Toro":
        fig.add_trace(
            ply_go.Scatter(
                x    = tf.range(interval[0], interval[1], Δx_ref, dtype=dtype)[4000:],
                y    = u[problema + ": referência"][0][4000:] ,
                mode = "lines"                                ,
                line = ply_go.scatter.Line(color="#666666")   ,
                name = "Reference"
            )
        )
    else:
        fig.add_trace(
            ply_go.Scatter(
                x    = tf.range(interval[0], interval[1], Δx_ref, dtype=dtype),
                y    = u[problema + ": referência"][0]      ,
                mode = "lines"                              ,
                line = ply_go.scatter.Line(color="#666666") ,
                name = "Reference"
            )
        )
        
    fig.update_yaxes(exponentformat="power")
#     fig.show()
    fig.write_html("htmls/" + problema[2:] + " WENO-ZD+.html")
#     fig.write_image("htmls/" + problema[2:] + " WENO-ZD+.png", width = 720, height = 480)

In [16]:
# Bloco de código vazio para evitar acidentes

In [10]:
# Salvando as soluções de referência
# referencia = {}
# for problema in problemas:
#     referencia[problema + ": referência"] = u[problema + ": referência"]

# with open('htmls/referência', 'wb') as file:
#     pickle.dump(referencia, file)

In [6]:
# Salvando as soluções obtidas
# sol = {}
# for problema in problemas:
#     for nome in nomes:
#         sol[problema + ": " + nome] = u[problema + ": " + nome]
        
# with open('htmls/soluções', 'wb') as file:
#     pickle.dump(sol, file)

In [9]:
Weights = {}

Weights[nomes[ 0]] = WENO_JS(   API, equation, γ=γ).Get_weights
Weights[nomes[ 1]] = WENO_JS_M( API, equation, γ=γ).Get_weights
Weights[nomes[ 2]] = WENO_JS_MS(API, equation, γ=γ).Get_weights
Weights[nomes[ 3]] = WENO_JS_BI(API, equation, γ=γ).Get_weights

Weights[nomes[ 4]] = WENO_Z(   API, equation, γ=γ).Get_weights
Weights[nomes[ 5]] = WENO_Z_M( API, equation, γ=γ).Get_weights
Weights[nomes[ 6]] = WENO_Z_MS(API, equation, γ=γ).Get_weights
Weights[nomes[ 7]] = WENO_Z_BI(API, equation, γ=γ).Get_weights

Weights[nomes[ 8]] = WENO_Zp(   API, equation, γ=γ).Get_weights
Weights[nomes[ 9]] = WENO_Zp_M( API, equation, γ=γ).Get_weights
Weights[nomes[10]] = WENO_Zp_MS(API, equation, γ=γ).Get_weights
Weights[nomes[11]] = WENO_Zp_BI(API, equation, γ=γ).Get_weights

Weights[nomes[12]] = WENO_ZC_M( API, equation, γ=γ).Get_weights
Weights[nomes[13]] = WENO_ZC_MS(API, equation, γ=γ).Get_weights
Weights[nomes[14]] = WENO_ZC_BI(API, equation, γ=γ).Get_weights

In [10]:
u_ref = {}
pesos = {}

for problema in problemas:
    
    init = cond_inicial(problema)
    
    fator     = init["fator"]
    Δx        = init["Δx"]
    fronteira = init["fronteira"]
    
    u_ref[problema] = u[problema + ": referência"]
#     u_ref[problema] = u_ref[problema][:,np.arange(0, len(u_ref[problema][0]), fator)]
    
    for nome in nomes:
        pesos[problema + ": " + nome] = Weights[nome](u_ref[problema], Δx, fronteira)[0][0]

In [11]:
for problema in problemas:
    
    init     = cond_inicial(problema)
    interval = init["interval"]
#     Δx       = init["Δx"]
#     aux_x    = tf.range(interval[0], interval[1], Δx, dtype=dtype)
    Δx_ref       = init["Δx_ref"]
    aux_x    = tf.range(interval[0], interval[1], Δx_ref, dtype=dtype)

    for nome in nomes:
        
        aux = problema + ": " + nome
        
        label_0 = tf.repeat([nome + ": omega 0"], [len(pesos[aux][:,0])])
        label_1 = tf.repeat([nome + ": omega 1"], [len(pesos[aux][:,1])])
        label_2 = tf.repeat([nome + ": omega 2"], [len(pesos[aux][:,2])])

        x     = tf.concat([          aux_x,           aux_x,           aux_x], axis = 0)
        y     = tf.concat([pesos[aux][:,0], pesos[aux][:,1], pesos[aux][:,2]], axis = 0)
        label = tf.concat([        label_0,         label_1,         label_2], axis = 0)
        
        df = pd.DataFrame(
            {
                "x"     : x     ,
                "y"     : y     ,
                "label" : label
            }
        )
        fig = ply_exp.line(
            data_frame = df    ,
            x          = x     ,
            y          = y     ,
            color      = label ,
            log_y      = True
        )
        fig.add_trace(
            ply_go.Scatter(
                x    = aux_x                                ,
                y    = np.exp(u_ref[problema][0])           ,
                mode = "lines"                              ,
                line = ply_go.scatter.Line(color="#666666") ,
                name = "Função",
            )
        )
        fig.update_yaxes(exponentformat="power")
#         fig.show()
        fig.write_html("htmls/Pesos para " + problema[2:] + " com " + nome + " WENO-D (referência).html")

ValueError: All arrays must be of the same length

In [49]:
for problema in problemas:
    
    init     = cond_inicial(problema)
    interval = init["interval"]
#     Δx       = init["Δx"]
#     aux_x    = tf.range(interval[0], interval[1], Δx, dtype=dtype)
    Δx_ref       = init["Δx_ref"]
    aux_x    = tf.range(interval[0], interval[1], Δx_ref, dtype=dtype)

    for i in np.arange(3):
        
        aux = problema + ": " + nomes[0]
        
        x     = aux_x
        y     = pesos[aux][:,i]
        label = tf.repeat(["omega " + str(i) + ": " + nomes[0]], [len(pesos[aux])])

        for j in range(1, len(nomes)):
            
            aux = problema + ": " + nomes[j]
            
            x     = tf.concat([    x, aux_x], axis = 0)
            y     = tf.concat([    y, pesos[aux][:,i]], axis = 0)
            label = tf.concat([label, tf.repeat(["omega " + str(i) + ": " + nomes[j]], [len(pesos[aux])])], axis = 0)

        df = pd.DataFrame(
            {
                "x"     : x     ,
                "y"     : y     ,
                "label" : label
            }
        )
        fig = ply_exp.line(
            data_frame = df    ,
            x          = x     ,
            y          = y     ,
            color      = label ,
            log_y      = True
        )
        fig.add_trace(
            ply_go.Scatter(
                x    = aux_x                                ,
                y    = np.exp(u_ref[problema][0])           ,
                mode = "lines"                              ,
                line = ply_go.scatter.Line(color="#666666") ,
                name = "Função"                             ,
            )
        )
        fig.update_yaxes(exponentformat="power")
#         fig.show()
        fig.write_html("htmls/Peso " + str(i) + " para " + problema[2:] + " WENO-D (referência).html")